In [1]:
import pandas as pd
import numpy as np 
import os
import sys
import fitz
import re
import pymongo

if os.path.abspath('../') not in sys.path:
    sys.path.insert(0, os.path.abspath('../'))

In [2]:
econ_input = '../data/raw/socio-economic-data-of-63-provinces-and-centrally-run-cities-2015-2021.pdf'
output_xlsx = '../data/processed/Viecon_data.xlsx'
features = [
    'AREA OF LAND (Thous. ha)',
    'POPULATION DENSITY (Person/km2)',
    'At current prices (Bill. dongs)',
    'State budget revenue (Bill. dongs)',
    'State budget expenditure (Bill. dongs)',
    'Investment at current prices (Bill. dongs)',
    'Area of floors of residential buildings constructed in the year by types of house (Thous. m2)',
    'POPULATION (Thous. pers.)',
    'Number of farms',
    'Planted area of cereals (Thous. ha)',
    'Number of schools (School)',
    'Production of fishery (Ton)',
    'Index of industrial production (%)', 
    'Retail sales of goods at current prices (Bill. dongs)',
    'Number of medical establishments (Esta.)',
] 
provinces = [
    'An Giang', 'Ba Ria-Vung Tau', 'Bac Giang', 'Bac Kan', 'Bac Lieu',
    'Bac Ninh', 'Ben Tre', 'Binh Dinh', 'Binh Duong', 'Binh Phuoc',
    'Binh Thuan', 'Ca Mau', 'Can Tho', 'Cao Bang', 'Da Nang', 'Dak Lak',
    'Dak Nong', 'Dien Bien', 'Dong Nai', 'Dong Thap', 'Gia Lai', 'Ha Giang',
    'Ha Nam', 'Ha Noi', 'Ha Tinh', 'Hai Duong', 'Hai Phong', 'Hau Giang',
    'Ho Chi Minh city', 'Hoa Binh', 'Hung Yen', 'Khanh Hoa', 'Kien Giang',
    'Kon Tum', 'Lai Chau', 'Lam Dong', 'Lang Son', 'Lao Cai', 'Long An',
    'Nam Dinh', 'Nghe An', 'Ninh Binh', 'Ninh Thuan', 'Phu Tho', 'Phu Yen',
    'Quang Binh', 'Quang Nam', 'Quang Ngai', 'Quang Ninh', 'Quang Tri',
    'Soc Trang', 'Son La', 'Tay Ninh', 'Thai Binh', 'Thai Nguyen',
    'Thanh Hoa', 'Thua Thien - Hue', 'Tien Giang', 'Tra Vinh',
    'Tuyen Quang', 'Vinh Long', 'Vinh Phuc', 'Yen Bai'
]

In [3]:
def extract_data_from_pdf(pdf_path, features_list, provinces_list):
    pdf_document = fitz.open(pdf_path)

    # Initialize a dictionary to store extracted data
    data_dict = {province: [] for province in provinces_list}

    # Iterate through each page in the PDF
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        
        page_text = page.get_text()

        # Find occurrences of provinces on the page
        occurrences = [(province, match.start()) for province in provinces_list for match in re.finditer(province, page_text)]

        # Iterate through each occurrence
        for province, start_index in occurrences:
            province_text = page_text[start_index:]
            next_province_index = min((occ[1] for occ in occurrences if occ[1] > start_index), default=len(page_text))
            province_text = province_text[:next_province_index]

            # Use regular expressions to extract data for all specified features
            features_data = {}
            for feature in features_list:
                # Construct the regular expression pattern based on the feature
                pattern = re.escape(feature) + r"\s+([\d.,\s]+)"

                # Find matches
                matches = re.findall(pattern, province_text)
                if matches:
                    features_data[feature] = list(map(float, matches[0].replace(',', '.').split()))

            # Append the extracted data to the corresponding province in the dictionary
            data_dict[province].append(features_data)

    # Close the PDF file
    pdf_document.close()

    return data_dict

# Extract data from PDF
result = extract_data_from_pdf('../data/raw/socio-economic-data-of-63-provinces-and-centrally-run-cities-2015-2021.pdf', features, provinces)
# Display the result
for province, occurrences_data in result.items():
    for index, features_data in enumerate(occurrences_data, start=1):
        if any(features_data.values()):
            print(f"{province} : {features_data}")


An Giang : {'AREA OF LAND (Thous. ha)': [353.7, 353.7, 353.6, 353.7, 353.7, 353.7, 353.7], 'POPULATION (Thous. pers.)': [2000.9, 1976.4, 1954.2, 1931.0, 1907.4, 1904.5, 1909.5]}
An Giang : {'POPULATION DENSITY (Person/km2)': [566.0, 559.0, 553.0, 546.0, 539.0, 538.0, 540.0]}
An Giang : {'At current prices (Bill. dongs)': [60466.8, 65466.9, 70719.7, 78152.9, 84939.6, 88959.2, 92237.9]}
An Giang : {'State budget revenue (Bill. dongs)': [13141.9, 14282.0, 16165.1, 19303.5, 20539.3, 21865.4, 21757.1], 'State budget expenditure (Bill. dongs)': [17687.6, 18002.0, 22522.4, 24880.1, 26701.5, 29402.7, 26782.3]}
An Giang : {'Investment at current prices (Bill. dongs)': [11228.7, 11920.3, 13488.6, 16146.9, 17695.7, 19804.4, 13186.2]}
An Giang : {'Number of farms': [697.0, 1180.0, 1063.0, 1016.0, 966.0, 878.0, 857.0], 'Planted area of cereals (Thous. ha)': [652.8, 677.1, 649.4, 630.6, 632.2, 642.9, 630.2]}
An Giang : {'Production of fishery (Ton)': [348079.0, 369843.0, 401724.0, 486804.0, 539909.0

In [4]:
our_dic = result
result_arr = []

for province, data_list in our_dic.items():
    for i in range(8):  # Modify the loop range to include 2021
        if i < 7:  # Exclude data for the year 2022
            row = {'province': province, 'year': 2015 + i}
            for entry in data_list:
                if entry and isinstance(entry, dict):
                    for key, value in entry.items():
                        if isinstance(value, list) and i < len(value):
                            if key == 'year':
                                if value[i] == "Sơ bộ Prel. 2021":
                                    row[key] = 2021
                                else:
                                    row[key] = value[i]
                            else:
                                row[key] = value[i] if value[i] is not None else np.nan
            result_arr.append(row)

Vietsociacono_data = pd.DataFrame(result_arr)

In [5]:
provinces = Vietsociacono_data['province'].value_counts().sort_index()
print(provinces.index)

Index(['An Giang', 'Ba Ria-Vung Tau', 'Bac Giang', 'Bac Kan', 'Bac Lieu',
       'Bac Ninh', 'Ben Tre', 'Binh Dinh', 'Binh Duong', 'Binh Phuoc',
       'Binh Thuan', 'Ca Mau', 'Can Tho', 'Cao Bang', 'Da Nang', 'Dak Lak',
       'Dak Nong', 'Dien Bien', 'Dong Nai', 'Dong Thap', 'Gia Lai', 'Ha Giang',
       'Ha Nam', 'Ha Noi', 'Ha Tinh', 'Hai Duong', 'Hai Phong', 'Hau Giang',
       'Ho Chi Minh city', 'Hoa Binh', 'Hung Yen', 'Khanh Hoa', 'Kien Giang',
       'Kon Tum', 'Lai Chau', 'Lam Dong', 'Lang Son', 'Lao Cai', 'Long An',
       'Nam Dinh', 'Nghe An', 'Ninh Binh', 'Ninh Thuan', 'Phu Tho', 'Phu Yen',
       'Quang Binh', 'Quang Nam', 'Quang Ngai', 'Quang Ninh', 'Quang Tri',
       'Soc Trang', 'Son La', 'Tay Ninh', 'Thai Binh', 'Thai Nguyen',
       'Thanh Hoa', 'Thua Thien - Hue', 'Tien Giang', 'Tra Vinh',
       'Tuyen Quang', 'Vinh Long', 'Vinh Phuc', 'Yen Bai'],
      dtype='object')


In [6]:
Vietsociacono_data.to_excel(output_xlsx, index=False)

In [7]:
# open the excel file 
Viecon = pd.read_excel(output_xlsx)
Viecon

,province,year,AREA OF LAND (Thous. ha),POPULATION (Thous. pers.),POPULATION DENSITY (Person/km2),At current prices (Bill. dongs),State budget revenue (Bill. dongs),State budget expenditure (Bill. dongs),Investment at current prices (Bill. dongs),Number of farms,Planted area of cereals (Thous. ha),Production of fishery (Ton),Index of industrial production (%),Retail sales of goods at current prices (Bill. dongs),Number of schools (School),Number of medical establishments (Esta.)
0,An Giang,2015,353.7,2000.9,566.0,60466.8,13141.9,17687.6,11228.7,697.0,652.8,348079.0,105.0,35835.0,547.0,191
1,An Giang,2016,353.7,1976.4,559.0,65466.9,14282.0,18002.0,11920.3,1180.0,677.1,369843.0,105.7,38093.0,542.0,192
2,An Giang,2017,353.6,1954.2,553.0,70719.7,16165.1,22522.4,13488.6,1063.0,649.4,401724.0,106.8,43129.0,536.0,3560
3,An Giang,2018,353.7,1931.0,546.0,78152.9,19303.5,24880.1,16146.9,1016.0,630.6,486804.0,108.9,47437.0,530.0,3807
4,An Giang,2019,353.7,1907.4,539.0,84939.6,20539.3,26701.5,17695.7,966.0,632.2,539909.0,109.9,51421.0,531.0,3969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,Yen Bai,2017,688.7,807.3,117.2,24917.8,14616.7,14413.5,10370.8,16.0,70.7,7497.0,106.9,12665.0,192.0,358
437,Yen Bai,2018,688.7,814.6,118.3,27590.4,16345.4,16025.7,11105.3,32.0,70.6,9017.0,104.3,13868.0,197.0,371
438,Yen Bai,2019,689.3,823.0,119.4,30530.1,18736.1,18480.1,13513.1,34.0,71.6,10485.0,110.9,15979.0,179.0,393
439,Yen Bai,2020,689.3,831.6,120.6,33414.8,20577.5,20472.8,15655.2,17.0,72.3,11634.0,108.1,16728.0,177.0,409


In [8]:
Viecon.columns 

Index(['province', 'year', 'AREA OF LAND (Thous. ha)',
       'POPULATION (Thous. pers.)', 'POPULATION DENSITY (Person/km2)',
       'At current prices (Bill. dongs)', 'State budget revenue (Bill. dongs)',
       'State budget expenditure (Bill. dongs)',
       'Investment at current prices (Bill. dongs)', 'Number of farms',
       'Planted area of cereals (Thous. ha)', 'Production of fishery (Ton)',
       'Index of industrial production (%)',
       'Retail sales of goods at current prices (Bill. dongs)',
       'Number of schools (School)',
       'Number of medical establishments (Esta.)'],
      dtype='object')

In [9]:
# transfer the POPULATION (Thous. pers.), Planted area of cereals (Thous. ha) into squared kilometers
Viecon['POPULATION (Thous. pers.)'] = Viecon['POPULATION (Thous. pers.)'] * 1000
Viecon['Planted area of cereals (Thous. ha)'] = Viecon['Planted area of cereals (Thous. ha)'] * 0.01
Viecon['AREA OF LAND (Thous. ha)'] = Viecon['AREA OF LAND (Thous. ha)'] * 0.01
Viecon

,province,year,AREA OF LAND (Thous. ha),POPULATION (Thous. pers.),POPULATION DENSITY (Person/km2),At current prices (Bill. dongs),State budget revenue (Bill. dongs),State budget expenditure (Bill. dongs),Investment at current prices (Bill. dongs),Number of farms,Planted area of cereals (Thous. ha),Production of fishery (Ton),Index of industrial production (%),Retail sales of goods at current prices (Bill. dongs),Number of schools (School),Number of medical establishments (Esta.)
0,An Giang,2015,3.537,2000900.0,566.0,60466.8,13141.9,17687.6,11228.7,697.0,6.528,348079.0,105.0,35835.0,547.0,191
1,An Giang,2016,3.537,1976400.0,559.0,65466.9,14282.0,18002.0,11920.3,1180.0,6.771,369843.0,105.7,38093.0,542.0,192
2,An Giang,2017,3.536,1954200.0,553.0,70719.7,16165.1,22522.4,13488.6,1063.0,6.494,401724.0,106.8,43129.0,536.0,3560
3,An Giang,2018,3.537,1931000.0,546.0,78152.9,19303.5,24880.1,16146.9,1016.0,6.306,486804.0,108.9,47437.0,530.0,3807
4,An Giang,2019,3.537,1907400.0,539.0,84939.6,20539.3,26701.5,17695.7,966.0,6.322,539909.0,109.9,51421.0,531.0,3969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,Yen Bai,2017,6.887,807300.0,117.2,24917.8,14616.7,14413.5,10370.8,16.0,0.707,7497.0,106.9,12665.0,192.0,358
437,Yen Bai,2018,6.887,814600.0,118.3,27590.4,16345.4,16025.7,11105.3,32.0,0.706,9017.0,104.3,13868.0,197.0,371
438,Yen Bai,2019,6.893,823000.0,119.4,30530.1,18736.1,18480.1,13513.1,34.0,0.716,10485.0,110.9,15979.0,179.0,393
439,Yen Bai,2020,6.893,831600.0,120.6,33414.8,20577.5,20472.8,15655.2,17.0,0.723,11634.0,108.1,16728.0,177.0,409


In [10]:
database_username = "admin01"
database_password = "LZRKfNnjX38rCOMs"
database_cluster_name = "bigdataforengineering.wef2a2d.mongodb.net"
connection_string = f"mongodb+srv://{database_username}:{database_password}@{database_cluster_name}/?retryWrites=true&w=majority"

new_database = 'processed_data'
economic = 'economic_indicators'

# Connect to MongoDB
client = pymongo.MongoClient(connection_string)

# Check if the connection was successful
if client.server_info():
    print("Connected to MongoDB successfully!")

# Access the specified database and collection
db = client[new_database]
collection = db[economic]

# Read the Excel file into a DataFrame
economic = pd.read_excel(output_xlsx)

# Save the DataFrame to MongoDB
collection.insert_many(economic.to_dict('records'))

# Verify if the data has been successfully inserted
cursor = collection.find({})
data_list = list(cursor)
updated_data = pd.DataFrame(data_list)
print("Data successfully inserted into MongoDB collection:")


Connected to MongoDB successfully!
Data successfully inserted into MongoDB collection:
